# Pinksy-Rinzel One Parameter of Freedom #

In [9]:
import pandas as pd
from pyomo.environ import *
from pyomo.core import *
from pyomo.dae import *
import numpy as np
from itertools import product
import pyomo.contrib.parmest.parmest as parmest
import scipy.io as spio
import matplotlib.pyplot as plt
import random

#Initializing data that we will model
#Model created using time step of 0.05 and runge-kutta 4th order method for approximating steps
treal = spio.loadmat('t_real.mat')
treal = [float(x) for x in treal['treal']]
#trealcut = [treal[i] for i in range(0, len(treal), 20)]
tpeaks = [float(treal[519]), treal[1544], treal[2569], treal[3594]]
#tpeaks = [25.95, 77.2, 128.45, 179.7]
#trealidx = trealcut.extend(tpeaks)
vreal = spio.loadmat('v_real.mat')
vreal = [float(x) for x in vreal['vreal']]
#vrealcut = [vreal[i] for i in range(0, len(vreal), 20)]
vpeaks = [vreal[i] for i in [519,1544,2569,3594]]

#hreal and nreal for help in computation
hreal = spio.loadmat('hreal.mat')
hreal = [float(x) for x in hreal['hreal']]
nreal = spio.loadmat('nreal.mat')
nreal = [float(x) for x in nreal['nreal']]

#breakpoints found using ppfit to fit piecewise polynomial to data w/o ODEs

#Simpler Model using Is=-0.5 to test if code works (works pretty well (works better w/ collocation))
#treals = spio.loadmat('treal_Is_0.5.mat')
#treals = [float(x) for x in treals.values()[0]]
#vreals = spio.loadmat('vreal_Is_0.5.mat')
#vreals = [float(x) for x in vreals.values()[0]]

#Model created using ode23 to determine times and estimate vs (as compared to runge-kutta w/ 0.05 time step)
#treal23 = spio.loadmat('t23.mat')
#treal23 = [float(x) for x in treal23.values()[0]]
#vreal23 = spio.loadmat('v23.mat')
#vreal23 = [float(x) for x in vreal23.values()[0]]
#tpeaks23 = [float(treal23[45]), float(treal23[136]), float(treal23[227]), float(treal23[320])]
#trealcut23 = [treal[i] for i in range(0, len(treal), 50)]

#breaks3 uses 200 linear (1st order) polynomials
breaks = spio.loadmat('breaks1600.mat')
breaks = [round(2*x,1) / 2 for x in breaks['breaks'][0]]
#rounding 2*x then dividing by 2 vs just rounding x in order to capture .05 second steps vs 0.1 second steps



def pr_model(mydata):
# **Creating Model**
    model = ConcreteModel()

#time (tt is total time)
    model.t = ContinuousSet(bounds=(0,200), initialize=treal) #200 ms time and breaks as finite element boundaries
#model.tt = Var(within=PositiveReals)

#constants
    Cm=3 # Membrane Capcitance uF/cm^2
    dt=0.05 # Time Step ms
    p = 0.5 #Fraction of cable length assigned to soma (1-p for dendtrite)
    Is=0.25 #External Current Applied #can be -0.5 for simpler model, can vary this as well and see what happens
    VNa=120 # mv Na reversal potential
    VK=-15 # mv K reversal potential
    VL=0 # mv Leakage reversal potential

#Parameters that we are solving for
    model.gNa = Var(bounds=(1.0,50.0), initialize=30.0) # mS/cm^2 Na max conductance (30 real)
    #model.gNa = Param(initialize=30)
    #model.gKDR = Var(bounds=(1.0,50.0), initialize=15) # mS/cm^2 K max conductance (15 real)
    model.gKDR = Param(initialize=15.0)
    #model.gL = Var(within=NonNegativeReals, bounds=(0,2))#, initialize=0.1) # mS/cm^2 Leakage max conductance (0.1 real)
    model.gL = Param(initialize=0.1)

#Variables
    model.Vs = Var(model.t, bounds=(-15,120), initialize=-4.6)
    model.h = Var(model.t, bounds=(0,1), initialize=0.001)
    model.n = Var(model.t, bounds=(0,1), initialize=0.999)

#Funcions
#forward rate constant for fast sodium
    def am(v):
        return 0.32*(13.1-v)/(exp((13.1-v)/4)-1)

#backward rate constant for fast sodium
    def bm(v):
        return 0.28*(v-40.1)/(exp((v-40.1)/5)-1)

#forward rate constant for DR activation
    def an(v):
        return 0.016*(35.1-v)/(exp((35.1-v)/5)-1)

#backward rate constant for DR activation
    def bn(v):
        return 0.25*exp(0.5-0.025*v)

#forward rate constant for sodium inactivation
    def ah(v):
        return 0.128*exp((17-v)/18)

#backward rate constant for sodium inactivation
    def bh(v):
        return 4/(1+exp((40-v)/5))


#somatic leak current
    def Ils(t):
        return model.gL*(model.Vs[t]-VL)

#steady-state sodium activation (instantaneous)
    def minf(t):
        return am(model.Vs[t])/(am(model.Vs[t])+bm(model.Vs[t]))

#sodium current (y(2) is h, inactivation of sodium current)
    def INa(t):
        return model.gNa*(minf(t)**2)*model.h[t]*(model.Vs[t]-VNa)

#delayed rectifier current (y(3) is n, activation of DR)
    def IKDR(t):
        return model.gKDR*model.n[t]*(model.Vs[t]-VK)

#initial values
#not using initial values greatly improves performance in terms of time and memory efficiency
#and reduces objective function by small factor
#(just using initial h and n values seems to be slightly more efficient and better than using none)
#initializing Vs values leads to memory issues and the solver failing due to memory constraints
    def condi(model):
        #yield model.Vs[0] == vreal[0] # Initial Membrane voltage
        #yield model.h[0] == hreal[0] # Initial h-value
        #yield model.n[0] == nreal[0] # Initial n-value
        yield ConstraintList.End
    #model.cond = ConstraintList(rule = condi) #try without initial conditions

#Defining derrivatives
    model.dVsdt = DerivativeVar(model.Vs, wrt=model.t)
    model.dhdt = DerivativeVar(model.h, wrt=model.t)
    model.dndt = DerivativeVar(model.n, wrt=model.t)

#Defining system governing ODE's (the model)
#Ode 2 and 3 are identical to Hodgskin-Huxley Model, Ode 1 is very similar
    def _ode1(mo, t):
        #if t == 0:
        #    return Constraint.Skip
        return mo.dVsdt[t] == (-Ils(t) - INa(t) - IKDR(t) + Is/p)/Cm
    model.ode1 = Constraint(model.t, rule=_ode1)

    def _ode2(mo, t):
        #if t == 0:
        #    return Constraint.Skip
        return mo.dhdt[t] ==  ah(mo.Vs[t])*(1-mo.h[t])-bh(mo.Vs[t])*mo.h[t]
    model.ode2 = Constraint(model.t, rule=_ode2)

    def _ode3(mo, t):
        #if t == 0:
        #    return Constraint.Skip
        return mo.dndt[t] == an(mo.Vs[t])*(1-mo.n[t])-bn(mo.Vs[t])*mo.n[t]
    model.ode3 = Constraint(model.t, rule=_ode3)
                                    
    
    #discretizer = TransformationFactory('dae.collocation')
    #discretizer.apply_to(model,nfe=5,ncp=3,scheme='LAGRANGE-RADAU')
    discretizer = TransformationFactory('dae.finite_difference')
    discretizer.apply_to(model,nfe=200, scheme='BACKWARD') 
#nfe value will be overridden if vector passed to be initialized in continuous set (model.t), is longer than value
#assigned to nfe (solver will tell you if this occurs)


#Array of finite elements
    fe = model.t.get_finite_elements()
#testing to see if we can initialize less values and get correct solution
    fecut = [fe[i] for i in range(0, len(fe), 2)]
#Initializing Vs values
#have to use nested for loop (as opposed to simple indexing) because of python (maybe pyomo) rounding error
    for ti in fe:
        for tn in treal:
            if abs(ti-tn) < 0.001: #accounting for rounding error
                model.Vs[ti].value = vreal[treal.index(tn)]
                #model.h[ti].value = hreal[treal.index(tn)]
                #model.n[ti].value = nreal[treal.index(tn)]

    def objfun(mo):
        myobj = 0
        for ti in fe:
            for tn in treal:
                if abs (ti-tn) < 0.001:
                    idx = treal.index(tn)
                    myobj += ((model.Vs[ti] - mydata[idx])**2)#*(vreal[idx]**4) #maybe use absolute value here too
        return myobj
    #model.obj = Objective(rule = objfun)
    
    return model

### Parameter estimation
# Vars to estimate
theta_names = ['gNa']
# Data
npd = np.array(vreal)
npdf = [float(x) for x in npd]
df = pd.DataFrame(npdf, index=treal, columns=['Vs'])
# Sum of squared error function
def SSE(mo,dat):
        myobj = 0
        feh = mo.t.get_finite_elements()
        for ti in feh:
            #for tn in treal:
             #   if abs (ti-tn) < 0.001:
                    idx = treal.index(ti)
                    myobj += ((mo.Vs[ti] - float(dat[idx]))**2)#*(vreal[idx]**4) #maybe use absolute value here too
        return myobj
    
pest = parmest.Estimator(pr_model, df, theta_names, SSE)

print (pest.theta_est())
#obj, theta = pest.theta_est()
#print(obj)
#print(theta)
### Parameter estimation with bootstrap resampling
'''
bootstrap_theta = pest.theta_est_bootstrap(50)
print(bootstrap_theta.head())
parmest.pairwise_plot(bootstrap_theta)
parmest.pairwise_plot(bootstrap_theta, theta, 0.8, ['MVN', 'KDE', 'Rect'])
### Likelihood ratio test
k1 = np.arange(0.78, 0.92, 0.02)
k2 = np.arange(1.48, 1.79, 0.05)
k3 = np.arange(0.000155, 0.000185, 0.000005)
theta_vals = pd.DataFrame(list(product(k1, k2, k3)), columns=theta_names)
obj_at_theta = pest.objective_at_theta(theta_vals)
print(obj_at_theta.head())
LR = pest.likelihood_ratio_test(obj_at_theta, obj, [0.8, 0.85, 0.9, 0.95])
print(LR.head())
parmest.pairwise_plot(LR, theta, 0.8)
'''

'''
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

tf = []
vf = []

for ti in sorted(model.t):
    tf.append(ti)
    vf.append(value(model.Vs[ti]))

#display parameter values
print('gL:')
print(value(model.gL))
print('gNa:')
print(value(model.gNa))
print('gKDR:')
print(value(model.gKDR))

#display times and values at peaks comapred to real values
tpks = []
vpks = []
tlen = len(tf)
for ti in range(tlen):
    if ti!=0 and ti!=(tlen-1):
        if vf[ti] > vf[ti-1] and vf[ti] > vf[ti+1]:
            tpks.append(tf[ti])
            vpks.append(vf[ti])

            
#plotting
plt.figure(1)
plt.subplot(1,2,1)
plt.plot(tf,vf)
plt.title('Voltage Results')
plt.xlabel('Time')
plt.ylabel('Voltage (mV)')
plt.subplot(1,2,2)
plt.plot(treal,vreal)
plt.title('Voltage Real')
plt.xlabel('Time')
plt.ylabel('Voltage (mV)')

print('Peak Times, Real Peak Times')
print (tpks, tpeaks)
print('Peak Values Estimated, Real Peak Values')
print (vpks, vpeaks)
print (vf[0])

#display finite elements
print('Finite Elements Boundaries')
print (fe)
print (len(fe))
print('Initialized Finite Elements')
print (fecut)
print (len(fecut))

dinfo = model.t.get_discretization_info()
print (dinfo)
'''



    of finite elements specified in apply. The larger number of finite
    elements will be used.
ERROR: Constructing component 'SecondStageCost' from data=None failed:
    KeyError: 0
ERROR: Failed to create model instance for scenario=Experiment0


KeyError: 0